## 参考资料：

[pandas 读取excel、一次性写入多个sheet、原有文件追加sheet](https://blog.csdn.net/qq_35318838/article/details/104692846)

[Pandas 获取EXCEL 各sheet name和内容的两种方法](https://blog.csdn.net/qq_35499652/article/details/120703525)

[python入门-爬取百度翻译中的双语例句](https://www.jianshu.com/p/92478814d217)


## 测试代码

In [110]:
import pandas as pd
import numpy as np

file = r"./RAW_data.xlsx"
data = pd.read_excel(file, sheet_name=0, header=2)

In [112]:
type(data)

pandas.core.frame.DataFrame

In [113]:
data.head()

,Term,Example of term used in context / Definition of term,Other (e.g. Title of source article/page/ section),Term.1,Example of term used in context / Definition of term.1,Other (e.g. Title of source article/page/ section).1
0,adult training program,NaN,NaN,NaN,NaN,NaN
1,application for payment,An application for payment is a type of docume...,NaN,付款申请书,NaN,NaN
2,capable of undertaking,NaN,NaN,NaN,NaN,NaN
3,dismissal,official notice that you have been fired from ...,NaN,开除，解雇,NaN,NaN
4,to be laid off,"If workers are laid off, they are told by thei...",NaN,解雇,NaN,NaN


## 正式代码

In [1]:
import requests
import json
import re
import execjs
import urllib
import time


f_english = open("english.txt", "w+")
f_chinese = open("chinese.txt", "w+")
f_synonym = open("synonym.txt", "w+")


def getTranslation(source = 'machine'):
    '''
    input:
        source: 待翻译的单词，句子
    output:
        res_zh      : str  中文结果
        liju_en_res : list 英文例句
        liju_zh_res : list 中文例句
    '''
    zh_res = ""
    liju_en_res = []
    liju_zh_res = []

    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36",
        "Cookie": "BAIDUID=4650B0B34048BBAA1E0B909B42F5A564:FG=1; BIDUPSID=4650B0B34048BBAA1E0B909B42F5A564; PSTM=1537177909; BDUSS=w0VmEzUFFWTTh0bld5VWVhNVo5MEEyV2ZKdTk3U2stMGZmWVQ1TTRuSnVkOHBiQVFBQUFBJCQAAAAAAAAAAAEAAAD0GzcNaG9uZ3F1YW4xOTkxAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAG7qoltu6qJbTk; pgv_pvi=6774493184; uc_login_unique=19e6fd48035206a8abe89f98c3fc542a; uc_recom_mark=cmVjb21tYXJrXzYyNDU4NjM%3D; MCITY=-218%3A; cflag=15%3A3; SIGNIN_UC=70a2711cf1d3d9b1a82d2f87d633bd8a02893452711; locale=zh; Hm_lvt_64ecd82404c51e03dc91cb9e8c025574=1539333192; from_lang_often=%5B%7B%22value%22%3A%22en%22%2C%22text%22%3A%22%u82F1%u8BED%22%7D%2C%7B%22value%22%3A%22zh%22%2C%22text%22%3A%22%u4E2D%u6587%22%7D%5D; REALTIME_TRANS_SWITCH=1; FANYI_WORD_SWITCH=1; HISTORY_SWITCH=1; SOUND_SPD_SWITCH=1; SOUND_PREFER_SWITCH=1; to_lang_often=%5B%7B%22value%22%3A%22zh%22%2C%22text%22%3A%22%u4E2D%u6587%22%7D%2C%7B%22value%22%3A%22en%22%2C%22text%22%3A%22%u82F1%u8BED%22%7D%5D; Hm_lpvt_64ecd82404c51e03dc91cb9e8c025574=1539333307",
    }

    # 获取网页源码
    html = requests.get('https://fanyi.baidu.com', headers=headers)
    html.encoding = 'utf-8'

    # 正则匹配 gtk
    matches = re.findall("window.gtk = '(.*?)';", html.text, re.S)
    for match in matches:
        gtk = match

    if gtk == "":
        print('Get gtk fail.')
        exit()
    # print('gtk = ' + gtk)

    # 正则匹配 token
    matches = re.findall("token: '(.*?)'", html.text, re.S)
    for match in matches:
        token = match

    if token == "":
        print('Get token fail.')
        exit()
    # print('token = ' + token)

    # 计算 sign
    signCode = 'function a(r,o){for(var t=0;t<o.length-2;t+=3){var a=o.charAt(t+2);a=a>="a"?a.charCodeAt(0)-87:Number(a),a="+"===o.charAt(t+1)?r>>>a:r<<a,r="+"===o.charAt(t)?r+a&4294967295:r^a}return r}var C=null;var hash=function(r,_gtk){var o=r.length;o>30&&(r=""+r.substr(0,10)+r.substr(Math.floor(o/2)-5,10)+r.substr(-10,10));var t=void 0,t=null!==C?C:(C=_gtk||"")||"";for(var e=t.split("."),h=Number(e[0])||0,i=Number(e[1])||0,d=[],f=0,g=0;g<r.length;g++){var m=r.charCodeAt(g);128>m?d[f++]=m:(2048>m?d[f++]=m>>6|192:(55296===(64512&m)&&g+1<r.length&&56320===(64512&r.charCodeAt(g+1))?(m=65536+((1023&m)<<10)+(1023&r.charCodeAt(++g)),d[f++]=m>>18|240,d[f++]=m>>12&63|128):d[f++]=m>>12|224,d[f++]=m>>6&63|128),d[f++]=63&m|128)}for(var S=h,u="+-a^+6",l="+-3^+b+-f",s=0;s<d.length;s++)S+=d[s],S=a(S,u);return S=a(S,l),S^=i,0>S&&(S=(2147483647&S)+2147483648),S%=1e6,S.toString()+"."+(S^h)}'

    sign = execjs.compile(signCode).call('hash', source, gtk)
    # print('source = ' + source + ', sign = ' + sign)

    # 请求接口
    fromLanguage = 'en'
    toLanguage = 'zh'

    # 请求接口地址
    v2transapi = 'https://fanyi.baidu.com/v2transapi?from=%s&to=%s&query=%s' \
                 '&transtype=translang&simple_means_flag=3&sign=%s&token=%s' % (
                     fromLanguage, toLanguage, urllib.parse.quote(source), sign, token)

    # print(v2transapi)

    #### 最大重试次数
    attempts = 0
    success = False
    while attempts < 3 and not success:
        try:
            translate_result = requests.get(v2transapi, headers=headers)
            success = True
        except:
            attempts += 1
            if attempts == 3:
                break

    result = json.loads(translate_result.text)
    # print(translate_result.text)

    # print("翻译结果:{}".format(result["trans_result"]["data"][0]["dst"]))
    try:
        zh_res =result["trans_result"]["data"][0]["dst"]
    
        # print(lines)
        ### line: 每一组句子，包含一句中文和一句英文
        ### sentences: 每个句子
        ### sentence: 句子中的每个词；格式：['The', 'w_0', 'w_0', 0, ' ']

        # print('-----双语例句-----')

        double = result["liju_result"]['double']
        if double != "":
            lines = json.loads(result["liju_result"]['double'])
            is_english = True
            for line in lines:
                # print(line)
                for sentences in line:
                    # print(sentences)
                    if isinstance(sentences, list):
                        s = ""
                        # print(sentences)
                        for i, sentence in enumerate(sentences): # 组词成句
                            if is_english:
                                if i > len(sentences) - 3:
                                    s += sentence[0]
                                else:
                                    s += sentence[0] + " "
                            else:
                                s += sentence[0]
                        
                        s.encode("gbk", 'ignore').decode("gbk", "ignore")
                        if is_english:
                            # print(s, file=f_english)
                            # print(s)
                            liju_en_res.append(s) #! 英文例句
                            is_english = False
                        else:
                            # print(s, file=f_chinese)
                            # print(s)
                            liju_zh_res.append(s) #! 中文例句
                            is_english = True

                        # print(s)
                        # print()
                        s = ""
    
    except:
        zh_res =""
        liju_en_res = []
        liju_zh_res =[]

    return dict(zh_res=zh_res, liju_en=liju_en_res, liju_zh=liju_zh_res)


f_vob = open("electronic.txt")
for line in f_vob:
    source = line
    res_dict = getTranslation(source)
    # print(res_dict)


f_vob.close()
f_english.close()
f_chinese.close()
f_synonym.close()


In [2]:
f_vob = ["adult training program","application for payment"]
for line in f_vob:
    source = line
    res_dict = getTranslation(source)
    print(res_dict)


{'zh_res': '', 'liju_en': [], 'liju_zh': []}
{'zh_res': '', 'liju_en': [], 'liju_zh': []}


In [47]:
import pandas as pd

data = pd.ExcelFile(r"./RAW_data.xlsx")
sheets =data.sheet_names
print(sheets)

['W1 - Employment', 'W2 & 3 - Housing', 'W4 - Welfare', 'W6 - Business-Finance', 'W7 - Business-Taxation', 'W9 - Tourism', 'W9 + W10 - Medicine- GPs']


In [48]:
for name in sheets:
    df_pre = data.parse(sheet_name= name)
    # print(type(df_pre))
    print(df_pre.shape)

(99, 6)
(99, 6)
(99, 6)
(101, 6)
(107, 6)
(102, 6)
(145, 6)


In [49]:
df1 = data.parse(sheet_name= sheets[0], header=2)
df1.head(3)

,Term,Example of term used in context / Definition of term,Other (e.g. Title of source article/page/ section),Term.1,Example of term used in context / Definition of term.1,Other (e.g. Title of source article/page/ section).1
0,adult training program,NaN,NaN,NaN,NaN,NaN
1,application for payment,An application for payment is a type of docume...,NaN,付款申请书,NaN,NaN
2,capable of undertaking,NaN,NaN,NaN,NaN,NaN


In [5]:
vobs =  ["adult training program","application for payment", "capable of undertaking"]
for vob in vobs:
    res_dict = getTranslation(vob)
    print(res_dict)

{'zh_res': '', 'liju_en': [], 'liju_zh': []}
{'zh_res': '', 'liju_en': [], 'liju_zh': []}
{'zh_res': '', 'liju_en': [], 'liju_zh': []}


In [59]:
res_dict.keys()

dict_keys(['zh_res', 'liju_en', 'liju_zh'])

In [61]:
res_dict['liju_zh']

['我们多次去函请求结清这笔账款而无任何结果，为此我们非常遗憾。',
 '将授权批准人批准的借款申请送出纳付款。',
 'SET协议是为解决信用卡支付业务在Internet上安全、可靠的支付而提出的一种解决电子商务的安全支付方案。',
 '应用程序提示用户输入安全支付信息。']

In [70]:
col = ["Term", "Ex1", "Ex2", "Term2", "Ex3","Ex4"]
txt = [1,2,3,4,5,6]

dict(zip(col,txt))

{'Term': 1, 'Ex1': 2, 'Ex2': 3, 'Term2': 4, 'Ex3': 5, 'Ex4': 6}

## 处理结果

In [13]:
import pandas as pd

res_df = pd.ExcelFile("./trans_res4.xlsx")
raw_df = pd.ExcelFile("./RAW_data.xlsx")


In [5]:
res_sheets = res_df.sheet_names
raw_sheets = res_df.sheet_names

In [8]:
from tqdm import trange

for i  in trange(len(res_sheets)):
    na_res = res_sheets[i]
    na_raw = raw_sheets[i]
    print(na_res==na_raw)

100%|██████████| 7/7 [00:00<00:00, 7013.89it/s]

True
True
True
True
True
True
True


In [21]:
name = res_sheets[i]
df_raw = raw_df.parse(sheet_name=name)
df_raw.iloc[2:, 2]
df_raw.head()

,Medicine - Working with GPs,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,English,NaN,NaN,LOTE,NaN,NaN
1,Term,Example of term used in context / Definition o...,Other (e.g. Title of source article/page/ sect...,Term,Example of term used in context / Definition o...,Other (e.g. Title of source article/page/ sect...
2,anaesthetic,a drug that makes a person or an animal unable...,NaN,麻醉的，麻醉剂，麻醉药,NaN,NaN
3,bandage,a strip of cloth used for tying around a part ...,NaN,绷带；用绷带包扎,NaN,NaN
4,diagnosis,the act of discovering or identifying the exac...,NaN,诊断；调查分析,NaN,NaN


In [23]:
writer = pd.ExcelWriter("./trans_res5.xlsx", mode="w")
for i  in trange(len(res_sheets)):
    name = res_sheets[i]
    df_res = res_df.parse(sheet_name=name)
    df_raw = raw_df.parse(sheet_name=name)

    df_raw.iloc[2:, 2] =  df_res.iloc[:, 2]
    df_raw.iloc[2:, 4] =  df_res.iloc[:, 4]
    df_raw.iloc[2:, 5] =  df_res.iloc[:, 5]

    df_raw.to_excel(writer, sheet_name=name, index=None)
    writer.save()

writer.close()


100%|██████████| 7/7 [00:00<00:00, 11.84it/s]
